- 목적: 1번 노트북이 저장한 Feather를 불러와 모델 학습/보정만 수행
- [수정] Data Leakage를 완벽히 수정한 K-Fold 로직 적용

In [ ]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import catboost as cb # 대부분이 범주형변수로 이루어진 데이터셋에서 예측 성능이 우수 => 범주형 변수를 one-hot encoding 등 encoding 작업을 하지 않고 그대로 모델의 input, 알아서 target encoding 해줌
from sklearn.model_selection import StratifiedKFold # 라벨 비율 유지하며 K-fold 분할
from sklearn.metrics import roc_auc_score, brier_score_loss, mean_squared_error
from sklearn.calibration import calibration_curve 
from sklearn.isotonic import IsotonicRegression # 확률 보정용 : 타겟 함수가 단조 증가함수일때만 가능
from tqdm import tqdm
import joblib # 모델/보정기 저장
import warnings

warnings.filterwarnings('ignore')
print("Big-Tech ML Engineer: K-Fold + Leakage Fix + Domain Features 최종 전략 시작.")

Big-Tech ML Engineer: K-Fold + Leakage Fix + Domain Features 최종 전략 시작.


## 1. 로컬 경로 설정 및 "전처리된 데이터" 로드

In [ ]:
BASE_DIR = "./data"
MODEL_SAVE_DIR = "./model" 
os.makedirs(MODEL_SAVE_DIR, exist_ok=True) # ./model 디렉토리 없으면 생성, exist_ok = True라 있어도 오류 X
FEATURE_SAVE_PATH = os.path.join(BASE_DIR, "all_train_data.feather")

try:
    print(f"전처리된 메인 피처 로드 중... (매우 빠름)")
    all_train_df = pd.read_feather(FEATURE_SAVE_PATH)

except FileNotFoundError as e:
    print(f"경고: {e}")
    print("먼저 1_Preprocess.ipynb를 실행하여 all_train_data.feather를 생성해야 합니다.")
    raise
print("데이터 로드 완료.")

전처리된 메인 피처 로드 중... (매우 빠름)
데이터 로드 완료.


In [ ]:
# Expected Calibration Error(ECE) 계산기 (버그 수정됨) 
def expected_calibration_error(y_true, y_prob, n_bins=10): # y_true: 실제 라벨(0/1), y_prob: 예측 확률(0~1 ideally), n_bins: 몇 개 구간으로 나눠서 calibration을 볼지.
    if len(y_true) == 0 or len(y_prob) == 0: return 0.0 # 데이터 없으면 ECE = 0
    y_prob = np.nan_to_num(y_prob, nan=0.0) # nan을 전부 0.0으로 치환
    
    df = pd.DataFrame({'y_true': y_true, 'y_prob': y_prob})
    
    bin_edges = np.linspace(0, 1, n_bins + 1) # 0~1 구간을 균등하게 n_bins개로 나누는 경계 생성.
    bin_edges[0] = -0.001 # 경계 살짝 벌려서 0.0/1.0 같은 값이 pd.cut에서 누락되지 않도록.
    bin_edges[-1] = 1.001 
    
    # pd.cut에서 NA가 발생하지 않도록 y_prob의 범위를 강제로 [0, 1]로 클리핑
    df['y_prob'] = np.clip(df['y_prob'], 0, 1)
    df['bin'] = pd.cut(df['y_prob'], bins=bin_edges, right=True) # 각 샘플을 예측 확률 기준으로 구간(bin)에 할당.
    
    # .groupby(..., observed=True) to handle potential empty bins
    bin_stats = df.groupby('bin', observed=True).agg(
        bin_total=('y_prob', 'count'), # 샘플 수
        prob_true=('y_true', 'mean'), # 실제 양성 비율
        prob_pred=('y_prob', 'mean') # 예측 확률 평균
    )
    
    non_empty_bins = bin_stats[bin_stats['bin_total'] > 0] # 샘플이 없는 bin 제거
    if len(non_empty_bins) == 0: return 0.0 # 전부 비어있으면 ECE = 0
    
    bin_weights = non_empty_bins['bin_total'] / len(y_prob) # 각 bin이 전체에서 차지하는 비율(가중치).
    prob_true = non_empty_bins['prob_true']
    prob_pred = non_empty_bins['prob_pred']
    
    ece = np.sum(bin_weights * np.abs(prob_true - prob_pred)) # ECE 정의 : Σ (bin_weight * |실제양성비 - 예측확률|)
    return ece

In [ ]:
# 최종 점수 계산기
def combined_score(y_true, y_prob): # AUC / Brier / ECE를 계산하고, 커스텀 스코어를 출력
    if len(y_true) == 0 or len(y_prob) == 0 or np.sum(y_true) == 0 or np.sum(y_true) == len(y_true): 
        print("  AUC: N/A (단일 클래스), Brier: N/A, ECE: N/A (No data)")
        return 1.0 
    # 데이터 없거나 모든 라벨이 0이거나 1인 경우(AUC 정의 안 됨) => "최악"으로 1.0 리턴 (이 스코어는 낮을수록 좋다는 설계이므로).
    
    y_prob = np.nan_to_num(y_prob, nan=0.0) # 다시 한 번 NaN 방지
    
    mean_auc = roc_auc_score(y_true, y_prob)
    mean_brier = mean_squared_error(y_true, y_prob)
    mean_ece = expected_calibration_error(y_true, y_prob) 
    score = 0.5 * (1 - mean_auc) + 0.25 * mean_brier + 0.25 * mean_ece # AUC는 클수록 좋으니 (1 - AUC). Brier / ECE는 작을수록 좋으니 그대로. 가중치 0.5 / 0.25 / 0.25.
    print(f"  AUC: {mean_auc:.4f}, Brier: {mean_brier:.4f}, ECE: {mean_ece:.4f}")
    print(f"  Combined Score: {score:.5f}")
    return score

In [ ]:
## 6. K-Fold 분리 (오류 수정)
print("\n[INFO] K-Fold 교차 검증 분리 시작...")
N_SPLITS = 5 # 5 교차 검증
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42) # Label 비율 유지 + 셔플 + 재현 가능한 분할

# fold별 train/val 인덱스를 저장할 리스트.
train_indices_list = []
val_indices_list = []

all_train_df['Label'] = all_train_df['Label'].fillna(0) # Label NaN 방지용

# skf.split(X, y)는 각 fold마다 train/val 인덱스를 반환. 이를 리스트에 순서대로 저장
for train_idx, val_idx in skf.split(all_train_df, all_train_df['Label']): 
    train_indices_list.append(train_idx)
    val_indices_list.append(val_idx)

print(f"{N_SPLITS}-Fold 분리 완료.")

CAT_FEATURES = ['Age', 'PrimaryKey'] # CatBoost에 “범주형으로 처리할 컬럼” 목록.

# [수정] 'Test' -> 'Test_x' (1_Preprocess에서 병합 시 생성된 이름)
DROP_COLS_TRAIN = ['Test_id', 'Test_x', 'Test_y', 'Label', 'TestDate', 'Year', 'Month', 'base_index']


[INFO] K-Fold 교차 검증 분리 시작...
5-Fold 분리 완료.


In [ ]:
## 7. Model A (CatBoost Only) - [수정] PK Stats 제거
def train_model_A(X_train, y_train, X_val, y_val, group_label="A"):
    
    # 피처 정의 (PK Stats 관련 피처 제외)
    base_cols = [col for col in X_train.columns if not col.startswith('pk_')] # X_train에 pk_ 통계 컬럼이 들어와도 사용하지 않도록 필터링
    numeric_cols = list(set(base_cols) - set(CAT_FEATURES) - set(DROP_COLS_TRAIN))
    
    cb_X_train = X_train[numeric_cols + CAT_FEATURES]
    cb_X_val = X_val[numeric_cols + CAT_FEATURES]
    
    for col in CAT_FEATURES:
        cb_X_train[col] = cb_X_train[col].fillna('nan').astype(str)
        cb_X_val[col] = cb_X_val[col].fillna('nan').astype(str)
        
    cat_features_indices = [cb_X_train.columns.get_loc(c) for c in CAT_FEATURES if c in cb_X_train] # CatBoost는 cat_features에 “컬럼 인덱스”를 넣어야 하므로 위치 계산
    
    print(f"\n[{group_label}] CatBoost (Base) 학습 시작... (피처 {len(cb_X_train.columns)}개)")
    cat_base_model = cb.CatBoostClassifier(
        iterations=3000,
        learning_rate=0.05, 
        depth=6, 
        l2_leaf_reg=3, 
        loss_function='Logloss', 
        eval_metric='AUC', # eval_metric='AUC': 검증 기준 AUC
        random_seed=42,
        thread_count=-1,
        early_stopping_rounds=100, 
        verbose=1000,
        task_type='GPU'
    )
    cat_base_model.fit(
        cb_X_train, y_train,
        eval_set=[(cb_X_val, y_val)],
        cat_features=cat_features_indices # cat_features_indices로 범주형 컬럼 지정.
    )
    
    print(f"\n[{group_label}] 단독 확률 보정 (Isotonic) 시작...")
    pred_cat_uncal = cat_base_model.predict_proba(cb_X_val)[:,1]

    print(f"[{group_label}] 비보정 단독 점수:") 
    _ = combined_score(y_val, pred_cat_uncal) # 보정 전 확률로 AUC/Brier/ECE 출력.
    
    # 단조 증가 형태로 raw_prob → calibrated_prob를 학습. out_of_bounds='clip': 학습 범위 밖 값은 양 끝으로 클리핑.
    final_calibrator = IsotonicRegression(y_min=0, y_max=1, out_of_bounds='clip')
    final_calibrator.fit(pred_cat_uncal, y_val) 
    
    pred_cat_calibrated = final_calibrator.predict(pred_cat_uncal)
    print(f"[{group_label}] ★보정된 단독★ 최종 점수:")
    _ = combined_score(y_val, pred_cat_calibrated)
    
    return cat_base_model, final_calibrator

In [ ]:
## 7. Model B (CatBoost Only)
def train_model_B(X_train, y_train, X_val, y_val, pk_stats_fold, group_label="B"):
    
    # --- [Leakage 수정] PK Stats 병합 ---
    X_train = X_train.merge(pk_stats_fold, on='PrimaryKey', how='left')
    X_val = X_val.merge(pk_stats_fold, on='PrimaryKey', how='left')
    
    # --- 피처 정의 ---
    numeric_cols = list(set(X_train.columns) - set(CAT_FEATURES) - set(DROP_COLS_TRAIN))
    common_numeric_cols = list(set(X_train[numeric_cols].columns) & set(X_val[numeric_cols].columns))
    
    cb_X_train = X_train[common_numeric_cols + CAT_FEATURES]
    cb_X_val = X_val[common_numeric_cols + CAT_FEATURES]
    
    for col in CAT_FEATURES:
        cb_X_train[col] = cb_X_train[col].fillna('nan').astype(str)
        cb_X_val[col] = cb_X_val[col].fillna('nan').astype(str)
        
    cat_features_indices = [cb_X_train.columns.get_loc(c) for c in CAT_FEATURES if c in cb_X_train]
    
    # --- CatBoost (Base) 학습 ---
    print(f"\n[{group_label}] CatBoost (Base) 학습 시작... (피처 {len(cb_X_train.columns)}개)")
    cat_base_model = cb.CatBoostClassifier(
        iterations=3000,
        learning_rate=0.05, 
        depth=6, 
        l2_leaf_reg=3, 
        loss_function='Logloss', 
        eval_metric='AUC',
        random_seed=42,
        thread_count=-1,
        early_stopping_rounds=100, 
        verbose=1000,
        task_type='GPU'
    )
    cat_base_model.fit(
        cb_X_train, y_train,
        eval_set=[(cb_X_val, y_val)],
        cat_features=cat_features_indices
    )
    
    # --- 보정기 학습 ---
    print(f"\n[{group_label}] 단독 확률 보정 (Isotonic) 시작...")
    pred_cat_uncal = cat_base_model.predict_proba(cb_X_val)[:,1]
    print(f"[{group_label}] 비보정 단독 점수:")
    _ = combined_score(y_val, pred_cat_uncal)
    
    final_calibrator = IsotonicRegression(y_min=0, y_max=1, out_of_bounds='clip')
    final_calibrator.fit(pred_cat_uncal, y_val) 
    
    pred_cat_calibrated = final_calibrator.predict(pred_cat_uncal)
    print(f"[{group_label}] ★보정된 단독★ 최종 점수:")
    _ = combined_score(y_val, pred_cat_calibrated)
    
    return cat_base_model, final_calibrator

In [ ]:
# K-Fold 루프 실행
all_pk_stats_folds = [] 
for fold in range(N_SPLITS):
    print(f"\n--- Fold {fold+1}/{N_SPLITS} 학습 시작 ---")
    
    train_idx = train_indices_list[fold]
    val_idx = val_indices_list[fold]
    
    # 1. 전체 데이터에서 Train/Val 인덱스로 분리
    train_df_fold = all_train_df.iloc[train_idx]
    val_df_fold = all_train_df.iloc[val_idx]

    # 2. [Leakage 수정] PK Stats를 'train_df_fold'로만 생성
    print(f"\n[Fold {fold+1}] K-Fold Target Encoding (PK Stats) 생성...")
    agg_funcs = {
        'Age_num': ['mean', 'min', 'max'], 'YearMonthIndex': ['mean', 'std', 'min', 'max'],
        'A1_rt_mean': ['mean', 'std'], 
        'A4_acc_congruent': ['mean', 'std'], 'A4_acc_incongruent': ['mean', 'std'], 'A4_stroop_rt_cost': ['mean', 'std'],
        'RiskScore': ['mean', 'std', 'max'], 
        'B1_change_acc': ['mean', 'std'], 'B1_nonchange_acc': ['mean', 'std'],
        'B3_rt_mean': ['mean', 'std'],
        'B4_flanker_acc_cost': ['mean', 'std'], 'B4_rt_mean': ['mean', 'std'],
        'RiskScore_B': ['mean', 'std', 'max'], 
        'Test_id': ['count']
    }

    valid_agg_funcs = {col: funcs for col, funcs in agg_funcs.items() if col in train_df_fold.columns}
    pk_stats_fold = train_df_fold.groupby('PrimaryKey').agg(valid_agg_funcs)
    pk_stats_fold.columns = ['_'.join(col).strip() for col in pk_stats_fold.columns.values]
    pk_stats_fold.rename(columns={'Test_id_count': 'pk_test_total_count'}, inplace=True)
    
    # [!!!!! KeyError 수정: 'Test_x' 사용 !!!!!]
    pk_test_type_count_fold = train_df_fold.groupby('PrimaryKey')['Test_x'].value_counts().unstack(fill_value=0)
    if 'A' not in pk_test_type_count_fold.columns:
        pk_test_type_count_fold['A'] = 0
    if 'B' not in pk_test_type_count_fold.columns:
        pk_test_type_count_fold['B'] = 0
    pk_test_type_count_fold = pk_test_type_count_fold[['A', 'B']]
    pk_test_type_count_fold.columns = ['pk_test_A_count', 'pk_test_B_count']
    pk_stats_fold = pk_stats_fold.join(pk_test_type_count_fold, how='left').reset_index()
    
    all_pk_stats_folds.append(pk_stats_fold) # Fold별 PK Stats 저장

    # 3. A모델용 데이터 생성
    # [!!!!! KeyError 수정: 'Test_x' 사용 !!!!!] => Test_x == 'A'인 샘플만 모아 Model A용.
    X_train_A = train_df_fold[train_df_fold['Test_x'] == 'A'].copy()
    y_train_A = X_train_A['Label'].values
    X_val_A = val_df_fold[val_df_fold['Test_x'] == 'A'].copy()
    y_val_A = X_val_A['Label'].values

    # 4. B모델용 데이터 생성 => Test_x == 'B'인 샘플만 모아 Model B용.
    X_train_B = train_df_fold[train_df_fold['Test_x'] == 'B'].copy()
    y_train_B = X_train_B['Label'].values
    X_val_B = val_df_fold[val_df_fold['Test_x'] == 'B'].copy()
    y_val_B = X_val_B['Label'].values

    # 5. 모델 A 학습 및 저장
    print("\n--- 모델 A (신규 자격) 학습 ---")
    cat_A, calibrator_A = train_model_A(X_train_A, y_train_A, X_val_A, y_val_A)
    
    joblib.dump(cat_A, os.path.join(MODEL_SAVE_DIR, f"catboost_A_fold{fold}.pkl"))
    joblib.dump(calibrator_A, os.path.join(MODEL_SAVE_DIR, f"calibrator_A_fold{fold}.pkl"))

    # 6. 모델 B 학습 및 저장 : B 샘플이 부족하거나 라벨이 단일 클래스면 학습 스킵.
    print("\n--- 모델 B (자격 유지) 학습 ---")
    if len(X_train_B) > 0 and len(X_val_B) > 0 and len(np.unique(y_train_B)) > 1:
        cat_B, calibrator_B = train_model_B(X_train_B, y_train_B, X_val_B, y_val_B, pk_stats_fold)
        
        joblib.dump(cat_B, os.path.join(MODEL_SAVE_DIR, f"catboost_B_fold{fold}.pkl"))
        joblib.dump(calibrator_B, os.path.join(MODEL_SAVE_DIR, f"calibrator_B_fold{fold}.pkl"))
    else:
        print(f"[{fold+1} Fold] B모델 학습/검증 데이터가 부족하여 이 Fold를 건너뜁니다.")
        joblib.dump(None, os.path.join(MODEL_SAVE_DIR, f"catboost_B_fold{fold}.pkl"))
        joblib.dump(None, os.path.join(MODEL_SAVE_DIR, f"calibrator_B_fold{fold}.pkl"))


--- Fold 1/5 학습 시작 ---

[Fold 1] K-Fold Target Encoding (PK Stats) 생성...

--- 모델 A (신규 자격) 학습 ---

[A] CatBoost (Base) 학습 시작... (피처 98개)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6355240	best: 0.6355240 (0)	total: 57.1ms	remaining: 2m 51s
bestTest = 0.7062878311
bestIteration = 862
Shrink model to first 863 iterations.

[A] 단독 확률 보정 (Isotonic) 시작...
[A] 비보정 단독 점수:
  AUC: 0.7063, Brier: 0.0202, ECE: 0.0008
  Combined Score: 0.15210
[A] ★보정된 단독★ 최종 점수:
  AUC: 0.7087, Brier: 0.0200, ECE: 0.0000
  Combined Score: 0.15065

--- 모델 B (자격 유지) 학습 ---

[B] CatBoost (Base) 학습 시작... (피처 126개)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6144319	best: 0.6144319 (0)	total: 18ms	remaining: 53.9s
bestTest = 0.7395755947
bestIteration = 73
Shrink model to first 74 iterations.

[B] 단독 확률 보정 (Isotonic) 시작...
[B] 비보정 단독 점수:
  AUC: 0.7396, Brier: 0.0385, ECE: 0.0224
  Combined Score: 0.14544
[B] ★보정된 단독★ 최종 점수:
  AUC: 0.7432, Brier: 0.0374, ECE: 0.0000
  Combined Score: 0.13774

--- Fold 2/5 학습 시작 ---

[Fold 2] K-Fold Target Encoding (PK Stats) 생성...

--- 모델 A (신규 자격) 학습 ---

[A] CatBoost (Base) 학습 시작... (피처 98개)


...

0:	test: 0.6025567	best: 0.6025567 (0)	total: 23.7ms	remaining: 1m 11s
bestTest = 0.733599335
bestIteration = 113
Shrink model to first 114 iterations.

[B] 단독 확률 보정 (Isotonic) 시작...
[B] 비보정 단독 점수:
  AUC: 0.7336, Brier: 0.0374, ECE: 0.0191
  Combined Score: 0.14734
[B] ★보정된 단독★ 최종 점수:
  AUC: 0.7365, Brier: 0.0364, ECE: 0.0000
  Combined Score: 0.14084

--- Fold 5/5 학습 시작 ---

[Fold 5] K-Fold Target Encoding (PK Stats) 생성...

--- 모델 A (신규 자격) 학습 ---

[A] CatBoost (Base) 학습 시작... (피처 98개)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6317332	best: 0.6317332 (0)	total: 20.8ms	remaining: 1m 2s
bestTest = 0.697182864
bestIteration = 332
Shrink model to first 333 iterations.

[A] 단독 확률 보정 (Isotonic) 시작...
[A] 비보정 단독 점수:
  AUC: 0.6972, Brier: 0.0209, ECE: 0.0012
  Combined Score: 0.15693
[A] ★보정된 단독★ 최종 점수:
  AUC: 0.6996, Brier: 0.0207, ECE: 0.0000
  Combined Score: 0.15541

--- 모델 B (자격 유지) 학습 ---

[B] CatBoost (Base) 학습 시작... (피처 126개)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6114148	best: 0.6114148 (0)	total: 17.9ms	remaining: 53.6s
bestTest = 0.7302920818
bestIteration = 111
Shrink model to first 112 iterations.

[B] 단독 확률 보정 (Isotonic) 시작...
[B] 비보정 단독 점수:
  AUC: 0.7303, Brier: 0.0379, ECE: 0.0267
  Combined Score: 0.15100
[B] ★보정된 단독★ 최종 점수:
  AUC: 0.7332, Brier: 0.0360, ECE: 0.0000
  Combined Score: 0.14242


In [9]:
# [신규] K-Fold로 생성된 PK Stats를 하나로 합침 (평균)
print("\n[INFO] K-Fold PK Stats 병합...")
if all_pk_stats_folds:
    all_pk_stats_df = pd.concat(all_pk_stats_folds)
    final_pk_stats = all_pk_stats_df.groupby('PrimaryKey').mean().reset_index()
    final_pk_stats_path = os.path.join(MODEL_SAVE_DIR, "pk_stats_final.csv")
    final_pk_stats.to_csv(final_pk_stats_path, index=False)
else:
    print("경고: 유효한 PK Stats가 생성되지 않았습니다. 빈 파일을 생성합니다.")
    pd.DataFrame().to_csv(os.path.join(MODEL_SAVE_DIR, "pk_stats_final.csv"), index=False)

print(f"\n[INFO] '최종 보정' CatBoost 모델 10개 및 최종 통계 피처 1개 저장 완료:")
print(f"  {final_pk_stats_path}")
print("Big-Tech ML Engineer: 미션 완료. 이 11개의 파일로 `submit.zip`을 재구성하십시오.")


[INFO] K-Fold PK Stats 병합...

[INFO] '최종 보정' CatBoost 모델 10개 및 최종 통계 피처 1개 저장 완료:
  ./model\pk_stats_final.csv
Big-Tech ML Engineer: 미션 완료. 이 11개의 파일로 `submit.zip`을 재구성하십시오.
